This script is the preliminary main operator script for running functions through the az-tilt estimation pipeline. To test out a function, we run the following steps:

1) Collect the data sets to test on.

2) Get metadata associated with the data sets, as this metadata is parameters into the function. FURTHER DISCUSSION: do we want to keep the naming conventions and passed parameters of the function consistent here?? This will definitely simplify things.

3) Get a list of all of the "submitted" modules that we want to run the datasets through. Each module contains the properly labeled function for the associated test (for example, all az-tilt tests require a function called run_az_tilt_estimation()).

4) Run function using the data set and the required metadata parameters and get outputs. Create a dictionary of outputs, containing results, ground truth outputs, and test type.

This is a rough draft that is expected to change.

In [39]:
# Import required packages

import pandas as pd
import glob
from importlib import import_module

In [40]:
# Read in all of the data sets and the associated metadata

data_files = glob.glob("./data/*.csv")
# Drop the metadata file
data_files = [x for x in data_files if 'metadata.csv' not in x]
print(data_files)

# Get the metadata file
metadata_df = pd.read_csv("./data/metadata.csv")

# Get a list of the modules that we're going to be testing. Ideally, this would be housed in some sort of database 
modules_to_import = ["az-tilt-estimator"]

# Generate a list to store all of the results as associated dictionary values
results_list = list()

['./data\\1201_ac_power_hw__2801.csv', './data\\1332_inv1_ac_power__2642.csv', './data\\1332_inv2_ac_power__2646.csv', './data\\1332_inv3_ac_power__2650.csv', './data\\4_ac_power__315.csv']


In [ ]:
# Loop through each file, get its associated metadata, and run it through the function
for file in data_files:
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    # Get the metadata associated with the specific file
    associated_metadata = dict(metadata_df[metadata_df['file'] == file].iloc[0])
    print(associated_metadata)
    # Build pandas series for power stream
    time_series = pd.Series(df[associated_metadata['power_stream']])
    # Import designated modules via importlib
    for mod in modules_to_import:
        mod = import_module(mod)
        function = getattr(mod, "run_az_tilt_estimation")
        tilt, az = function(time_series,
                           associated_metadata['latitude'],
                           associated_metadata['longitude'],
                           associated_metadata['time_zone'])
        print(tilt,az)
        results_link.append({"test": "azimuth-tilt estimation",
                            "data": file,
                            "module": mod,
                            "outputs": (tilt, az),
                            "ground_truth": (associated_metadata['tilt'], associated_metadata['azimuth'])})
    

In [38]:
print(df)

                     ac_power_hw__2801
measured_on                           
2015-01-01 00:03:00               -2.0
2015-01-01 00:06:00               -2.0
2015-01-01 00:09:00               -2.0
2015-01-01 00:12:00               -2.0
2015-01-01 00:15:00               -2.0
...                                ...
2018-06-30 23:35:00               -2.0
2018-06-30 23:40:00               -2.0
2018-06-30 23:45:00               -2.0
2018-06-30 23:50:00               -2.0
2018-06-30 23:55:00               -2.0

[442011 rows x 1 columns]
